In [1]:
import sqlite3
import pandas as pd

### Create empty database file and empty table

In [17]:
!rm ../data/velib.sqlite

rm: cannot remove '../data/velib.sqlite': No such file or directory


In [18]:
!touch ../data/velib.sqlite

In [19]:
conn = sqlite3.connect('../data/velib.sqlite')
c = conn.cursor()

In [20]:
c.execute("""
CREATE TABLE stations (datetime text, stationCode int, meca int, elec int, park int)
""")

### Fetch CSV file and clean it

In [10]:
stations = pd.read_csv('../data/data-5m.csv', low_memory=False)

In [12]:
stations['datetime'] = pd.to_datetime(stations.datetime, format='%Y-%m-%d %H-%M')
for col in ['stationCode', 'meca', 'elec', 'park']:
    stations[col] = pd.to_numeric(stations[col], errors='coerce')
stations = stations.dropna()
for col in ['stationCode', 'meca', 'elec', 'park']:
    stations[col] = stations[col].astype(int)

In [13]:
stations.dtypes

datetime       datetime64[ns]
stationCode             int64
meca                    int64
elec                    int64
park                    int64
dtype: object

In [14]:
stations.describe()

,stationCode,meca,elec,park
count,3.130036e+07,3.130036e+07,3.130036e+07,3.130036e+07
mean,1.831298e+04,7.507661e+00,3.584884e+00,1.840734e+01
std,1.205379e+04,9.378114e+00,3.654926e+00,1.230630e+01
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.101700e+04,1.000000e+00,1.000000e+00,9.000000e+00
50%,1.603200e+04,4.000000e+00,3.000000e+00,1.700000e+01
75%,2.131900e+04,1.100000e+01,5.000000e+00,2.600000e+01
max,9.200800e+04,7.400000e+01,5.900000e+01,7.100000e+01


### Convert pandas table to sqlite table and test it

In [21]:
stations.to_sql('stations', conn, if_exists='append', index=False)

In [45]:
c.execute("""
SELECT * FROM stations
""")

In [46]:
c.fetchone()

('2021-10-15 06:11:00', 16107, 2, 4, 28)

### Queries

In [47]:
query = """
SELECT s.datetime, s.meca + s.elec AS bikes, s.park
FROM stations s
WHERE s.stationCode = 12001
"""

In [48]:
pd.read_sql_query(query, conn).head()

,datetime,bikes,park
0,2021-10-15 06:11:00,16,44
1,2021-10-15 06:15:00,16,44
2,2021-10-15 06:20:00,16,44
3,2021-10-15 06:25:00,16,44
4,2021-10-15 06:30:00,16,44
